In [ ]:
import os
import sys
import csv
import glob
import math
import argparse
from typing import Tuple, List, Dict

import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch import nn, Tensor
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

In [ ]:
batch_size = 64
num_workers = 12
max_iter = 200000
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomChoice([
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
        transforms.RandomResizedCrop(224),
        transforms.RandomAffine(
            degrees=15, translate=(0.2, 0.2),
            scale=(0.8, 1.2), shear=15, resample=Image.BILINEAR)
    ]),
    transforms.ToTensor(),
    transforms.Normalize((0.4452, 0.4457, 0.4464), (0.2592, 0.2596, 0.2600)),
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4452, 0.4457, 0.4464), (0.2592, 0.2596, 0.2600)),
])

In [ ]:
class LandmarkDataset(Dataset):
    def __init__(self, mode: str = 'train', transforms: transforms = None):
        self.mode = mode
        self.image_ids = glob.glob(f'./data/{mode}/**/**/*')
        if self.mode == 'train':
            with open('./data/train.csv') as f:
                labels = list(csv.reader(f))[1:]
                self.labels = {label[0]: int(label[1]) for label in labels}

        self.transforms = transforms

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image = Image.open(self.image_ids[index]).convert('RGB')
        image_id = os.path.splitext(os.path.basename(self.image_ids[index]))[0]
        if self.transforms is not None:
            image = self.transforms(image)
        
        if self.mode == 'train':
            label = self.labels[image_id]
            return image, label
        else:
            return image_id, image

In [ ]:
trainset = LandmarkDataset('train', transforms_train)
testset = LandmarkDataset('test', transforms_test)

train_loader = DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(
    testset, batch_size=16, shuffle=False, num_workers=num_workers)

model = models.resnet101(pretrained=True)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

In [ ]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val: float, n: int = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def gap(preds: Tensor, confs: Tensor, targets: Tensor) -> float:
    assert len(preds.shape) == 1
    assert len(confs.shape) == 1
    assert len(targets.shape) == 1
    assert preds.shape == confs.shape and confs.shape == targets.shape

    _, indices = torch.sort(confs, descending=True)

    confs = confs.cpu().numpy()
    preds = preds[indices].cpu().numpy()
    targets = targets[indices].cpu().numpy()

    res, true_pos = 0.0, 0

    for i, (c, p, t) in enumerate(zip(confs, preds, targets)):
        rel = int(p == t)
        true_pos += rel

        res += true_pos / (i + 1) * rel

    res /= targets.shape[0]

    return res

In [ ]:
def train(
    model: nn.Module, data_loader: DataLoader, criterion: nn.Module, 
    optimizer: optim, scheduler: optim.lr_scheduler, device: torch.device):
    
    epoch_size = len(trainset) // batch_size
    num_epochs = math.ceil(max_iter / epoch_size)
    
    iteration = 0
    losses = AverageMeter()
    scores = AverageMeter()
    corrects = AverageMeter()
    
    model.train()
    for epoch in range(num_epochs):
        if (epoch+1)*epoch_size < iteration:
            continue
            
        if iteration == max_iter:
            break
            
        correct = 0
        for inputs, targets in data_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            confs, preds = torch.max(outputs.detach(), dim=1)
            
            optimizer.zero_grad()
            
            loss.backward()
            optimizer.step()

            losses.update(loss.item(), inputs.size(0))
            scores.update(gap(preds, confs, targets), inputs.size(0))
            corrects.update((preds == targets).float().sum(), input_size(0))
            
            iteration += 1

            if iteration % args.verbose_eval == 0:
                print(f'[{epoch+1}/{iteration}] Loss: {losses.val:.4f}' \
                      f' Acc: {corrects.val:.4f} GAP: {scores.val:.4f}')
         
            if iteration in [20000, 70000, 140000]:
                scheduler.step()

In [ ]:
def test(model: nn.Module, data_loader: DataLoader, device: torch.device):
    submission = pd.read_csv('./data/sample_submission.csv', index_col='id')
                             
    model.eval()
    for image_id, inputs in data_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        outputs = nn.Softmax(dim=1)(outputs)

        landmark_ids = np.argmax(outputs, axis=1)
        confidence = outputs[0, landmark_ids]
        submission.loc[image_id, 'landmark_id'] = landmark_ids
        submission.loc[image_id, 'conf'] = confidence
        
    submission.to_csv('submission.csv')